In [8]:
import os
import pickle
import time
from pprint import pprint
from typing import Any, Dict, List

import datasets
import kscope
from datasets import Dataset
from kscope import Model
from tqdm import tqdm
from transformers import AutoTokenizer

In [4]:
# Establish a client connection to the kscope service
client = kscope.Client(gateway_host="llm.cluster.local", gateway_port=3001)
client.model_instances

[{'id': 'b11f3264-9c03-4114-9d56-d39a0fa63640',
  'name': 'OPT-175B',
  'state': 'ACTIVE'},
 {'id': 'c5ea8da7-3384-4c7b-b47f-95ed1a485897',
  'name': 'OPT-6.7B',
  'state': 'ACTIVE'}]

In [5]:
model = client.load_model("llama2-13b")
# If this model is not actively running, it will get launched in the background.
# In this case, wait until it moves into an "ACTIVE" state before proceeding.
while model.state != "ACTIVE":
    time.sleep(1)

### Activation Generation 

In this notebook, we're going to extract the activations from the final non-pad token of LLaMA-13B for different sets of text inputs. This is done (on the back-end) by inserting hooks into the model that allow for extraction of the model's intermediate latent representations. In this notebook, we'll vary both the layer we extract information from and the input itself to consider the affect that these choices have on the performance of a downstream task. In this case, we'll consider a sampling of the IMDB sentiment analysis task to probe these choices.

To start, we need to define a configuration for the model generation. Because we only care about the activations of our input, the configuration is less important. The only thing we really need to do is set the `max_tokens` to 1 so that we don't have to worry about indexing into the right spot in our activation matrix. That is, the activations we care about will just occur in the last slot of the tensor. For a discussion of the configuration parameters see [CONFIG_README.md](../../prompting_vector_llms/CONFIG_README.md).

In [6]:
generation_config = {"max_tokens": 1}

### Activation Generation 

Activation generation is quite easy. We can use the client to query the remote model and explore the various modules. Here, we are listing only the last 10 layers of the model.

In [7]:
model.module_names[-10:]

['decoder.layers.95.self_attn',
 'decoder.layers.95.self_attn.dropout_module',
 'decoder.layers.95.self_attn.qkv_proj',
 'decoder.layers.95.self_attn.out_proj',
 'decoder.layers.95.self_attn_layer_norm',
 'decoder.layers.95.fc1',
 'decoder.layers.95.fc2',
 'decoder.layers.95.final_layer_norm',
 'decoder.layer_norm',
 'decoder.output_projection']

We can select the module names of interest and pass them into a `get_activations` function alongside our set of prompts.

In [6]:
prompts = ["Hello World", "Fizz Buzz"]

module_name = "output"

activations = model.get_activations(prompts, [module_name], generation_config)
pprint(activations)

# We sent a batch of 2 prompts to the model.
# So a list of length two is returned containing activations for the requested layer
for activations_single_prompt in activations.activations:
    # For each prompt we extract the activations associated with the target module.
    raw_activations = activations_single_prompt[module_name]
    # The activations should have shape (number of tokens + 1) x (activation size)
    # For example, LLaMA-13B has an embedding dimension for the layer requested of 4096
    print("Tensor Shape:", raw_activations.shape)

Activations(activations=[{'decoder.layers.95.fc2': tensor([[ 0.0062, -0.4624,  0.4702,  ...,  0.0985,  1.2588, -0.7002],
        [-0.2197,  0.3364, -0.1965,  ..., -0.2651,  0.0475, -0.1638]],
       dtype=torch.float16)}, {'decoder.layers.95.fc2': tensor([[ 2.5312, -1.4609, -1.7266,  ...,  1.9453, -0.8262, -1.5234],
        [-0.2294, -0.6396,  0.9043,  ..., -0.8140,  0.1183,  0.2437],
        [-0.8062,  0.8320, -0.8003,  ..., -0.8530, -0.7656,  0.5718]],
       dtype=torch.float16)}], logprobs=[[None, -7.15677547454834, -6.544065475463867], [None, -5.25247859954834, -6.673819065093994, -5.898566246032715]], text=['Hello World', 'Fizz Buzz'], tokens=[['</s>', 'Hello', ' World'], ['</s>', 'F', 'izz', ' Buzz']])
Tensor Shape: torch.Size([2, 12288])
Tensor Shape: torch.Size([3, 12288])


__NOTE__: In the code below, we're going to only use batch sizes of 1 to ensure memory management on the backend doesn't get out of hand and slow the model down.

In [4]:
# Tokenizer prepares the input of the model. LLaMA models of all sizes use the same underlying tokenizer
tokenizer = AutoTokenizer.from_pretrained("/Users/david/Desktop/LLaMA2_Tokenizer")
# Let's test out how the tokenizer works on an example sentence. Note that the token with ID = 1 is the
# Beginning of sentence token ("BOS")
encoded_tokens = tokenizer.encode("Fizz Buzz")
print(f"Encoded Tokens: {encoded_tokens}")
# If you ever need to move back from token ids, you can use tokenizer.decode or tokenizer.batch_decode
decoded_tokens = [tokenizer.decode(encoded_token) for encoded_token in encoded_tokens]
print(f"Decoded Tokens: {decoded_tokens}")

Encoded Tokens: [1, 383, 4981, 350, 18813]
Decoded Tokens: ['<s>', 'F', 'izz', 'B', 'uzz']


As can be seen above, the second example, "Fizz Buzz", is tokenized as ["F", "izz", "B", 'uzz]. So we receive a tensor with 5 rows (one for each token and a final one for the next token to be generated) and 4096 columns (the hidden dimension of LLaMA-2-13B).

As a proof of concept of the few-shot abilities of LLMs, we'll only use a small training dataset and will only perform validation using a small test subset for compute efficiency.

* Training set: 100 randomly sampled training examples
* Test set: 300 randomly sample test examples

In [6]:
imdb = datasets.load_dataset("imdb")
train_size = 100
test_size = 300
n_demonstrations = 5

activation_save_path = "./resources/"

small_train_dataset = imdb["train"].shuffle(seed=42).select([i for i in list(range(train_size))])
small_test_dataset = imdb["test"].shuffle(seed=42).select([i for i in list(range(test_size))])
# We're going to be experimenting with the affect that prompting the model for the task we care about has on a
# classifier trained on the activations in terms of performance. So we will construct demonstrations by randomly
# selecting a set of 5 examples from the training set to serve this purpose.
small_demonstration_set = imdb["train"].shuffle(seed=42).select([i for i in list(range(n_demonstrations))])

In [9]:
# Split a list into a tuple of lists of a fixed size
def batcher(prompts: List[str], batch_size: int) -> Dataset:
    return (prompts[pos : pos + batch_size] for pos in range(0, len(prompts), batch_size))

In [ ]:
# We're running a lot of activation retrievals. Once in a while there is a json decoding or triton error. If that
# happens, we retry the activations request.
def get_activations_with_retries(prompt: str, layers: List[str], config: Dict[str, Any], retries: int = 5) -> Any:
    for _ in range(retries):
        try:
            return model.get_activations(prompt, layers, config)
        except Exception as e:  # noqa: F841
            print("Something went wrong in activation retrieval...retrying")
    raise ValueError("Exceeded retry limit. Exiting Process")

### Raw Text Activations

Let's start by getting the activations associated with the raw review text. We'll do activations for the text coupled with a prompt below.

In [9]:
def generate_dataset_activations(
    split: str,
    inputs: List[str],
    labels: List[int],
    model: Model,
    module_name: str,
    pickle_name: str,
    batch_size: int = 1,
) -> None:
    print("Generating Activations with Prompts: " + split)

    parsed_activations = []
    for input_batch in tqdm(batcher(inputs, batch_size)):
        # Getting activations for each input batch. For an example of how get_activations works, see beginning of this
        # notebook.
        raw_activations = model.get_activations(input_batch, [module_name], generation_config).activations
        for raw_activation in raw_activations:
            # We will be performing classification on the last token non-pad token of the sequence. This is common
            # practice for autoregressive models (e.g. OPT, Falcon, LLaMA-2). So we only keep the last row of the
            # activation matrix.
            parsed_activations.append(raw_activation[module_name][-1].float())

    cached_activations = {"activations": parsed_activations, "labels": labels}

    with open(os.path.join(activation_save_path, f"{split}{pickle_name}.pkl"), "wb") as handle:
        pickle.dump(cached_activations, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [10]:
module_names = ["output", "output", "output", "output"]
layer_numbers = ["10", "20", "30", "39"]

train_labels = small_train_dataset["label"]
test_labels = small_test_dataset["label"]

assert len(module_names) == len(layer_numbers)

for modulel_name, layer_number in zip(module_name, layer_numbers):
    generate_dataset_activations(
        "train", small_train_dataset["text"], train_labels, model, module_name, f"_activations_demo_{layer_number}"
    )
    generate_dataset_activations(
        "test", small_test_dataset["text"], test_labels, model, module_name, f"_activations_demo_{layer_number}"
    )

Generating Activations with Prompts: train
Batch Number 0 Complete
Batch Number 1 Complete
Batch Number 2 Complete
Batch Number 3 Complete
Batch Number 4 Complete
Batch Number 5 Complete
Batch Number 6 Complete
Generating Activations with Prompts: test
Batch Number 0 Complete
Batch Number 1 Complete
Batch Number 2 Complete
Batch Number 3 Complete
Batch Number 4 Complete
Batch Number 5 Complete
Batch Number 6 Complete
Batch Number 7 Complete
Batch Number 8 Complete
Batch Number 9 Complete
Batch Number 10 Complete
Batch Number 11 Complete
Batch Number 12 Complete
Batch Number 13 Complete
Batch Number 14 Complete
Batch Number 15 Complete
Batch Number 16 Complete
Batch Number 17 Complete
Batch Number 18 Complete


### Prompt Conditioned Activations

Now let's generate activations pre-conditioned with an instruction and a few demonstrations.

In [11]:
def create_demonstrations(instruction: str, demonstration_set: Dataset) -> str:
    label_int_to_str = {0: "negative", 1: "positive"}
    demonstration = f"{instruction}"
    demo_texts = demonstration_set["text"]
    demo_labels = demonstration_set["label"]
    for text, label in zip(demo_texts, demo_labels):
        # truncate the text in case it is very long (cutting the first part of text)
        split_text = text.split(" ")
        if len(split_text) > 64:
            text = " ".join(split_text[-128:])
        demonstration = f"{demonstration}\nText: {text}\nSentiment: {label_int_to_str[label]}"
    return f"{demonstration}\n"

In [12]:
def create_prompts(texts: List[str], demonstration: str) -> List[str]:
    return [f"{demonstration}Text: {text} The sentiment is" for text in texts]

Below we show the demonstration structure (based on 5 examples) and what each prompt passed to OPT looks like

In [13]:
demonstration = create_demonstrations("Classify the sentiment of the text.", small_demonstration_set)
print(f"Demonstration:\n{demonstration}")

train_prompts = create_prompts(small_train_dataset["text"], demonstration)
test_prompts = create_prompts(small_test_dataset["text"], demonstration)
print(f"Prompt Example:\n{train_prompts[0]}")

Demonstration:
Classify the sentiment of the text.

Text: There is no relation at all between Fortier and Profiler but the fact that both are police series about violent crimes. Profiler looks crispy, Fortier looks classic. Profiler plots are quite simple. Fortier's plot are far more complicated... Fortier looks more like Prime Suspect, if we have to spot similarities... The main character is weak and weirdo, but have "clairvoyance". People like to compare, to judge, to evaluate. How about just enjoying? Funny thing too, people writing Fortier looks American but, on the other hand, arguing they prefer American series (!!!). Maybe it's the language, or the spirit, but I think this series is more English than American. By the way, the actors are really good and funny. The acting is not superficial at all... The sentiment is positive.

Text: a great. The plot is very true to the book which is a classic written by Mark Twain. The movie starts of with a scene where Hank sings a song with a 

In [ ]:
module_names = ["output", "output", "output", "output"]
layer_numbers = ["10", "20", "30", "39"]

train_labels = small_train_dataset["label"]
test_labels = small_test_dataset["label"]

assert len(module_names) == len(layer_numbers)

for modulel_name, layer_number in zip(module_name, layer_numbers):
    generate_dataset_activations(
        "train", train_prompts, train_labels, model, module_name, f"_activations_with_prompts_demo_{layer_number}"
    )
    generate_dataset_activations(
        "test",
        test_prompts,
        test_labels,
        model,
        module_name,
        f"_activations_with_prompts_demo_{layer_number}",
    )

With these activations saved, the next step is to train a simple classifier on top of them in order to perform the sentiment classification. This is done in the `train_on_activations.ipynb` notebook.